In [1]:
# Cómo importar las librerías
import xlwings  as xw
import numpy    as np
import pandas   as pd
import datetime as dt
import os
import sys
from pathlib import Path
import re

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
plt.style.use('fivethirtyeight')

#from statsmodels.graphics.tsaplots import plot_acf
#from statsmodels.graphics.tsaplots import plot_pacf


# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

# Configuracion Pandas
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
libro = xw.Book('GGastos_v060422.xlsm')
periodo = libro.sheets('Menu').range('H2').value
mes  = periodo[0:2]
anio = periodo[2:]
carpeta = Path(r'G:\Mi unidad\Mis_Pagos', anio, periodo)
Archivo = list(carpeta.glob('CA*$*0020382809*.xls'))

In [3]:
Archivo

[WindowsPath('G:/Mi unidad/Mis_Pagos/2022/022022/CA $ 900 0020382809-Movimientos.xls')]

In [4]:
datos = pd.read_excel(Archivo[0], names=['Fecha', 'Concepto', 'Descripcion', 'Monto', 'NroComprobante'], 
                        skiprows=5, skipfooter=2, engine='xlrd')  
datos['Fecha'] = pd.to_datetime(datos['Fecha'], format='%d/%m/%Y')

datos['Monto'] = datos['Monto'].str.lstrip('$')
datos['Monto'] = datos['Monto'].str.strip()
datos['Monto'] = datos['Monto'].str.replace('.', '',  regex=False)  # Elimina los puntos de miles
datos['Monto'] = datos['Monto'].str.replace(',', '.', regex=False)  # Reemplaza la coma decimal por el punto decimal
datos['Monto'] = pd.to_numeric(datos['Monto'])  # Pasa de type Object a Float
    #datos['Monto'] = datos['Monto'].astype(float)

datos['NroComprobante'] = datos['NroComprobante'].astype(str)

archivo_cp = Path(r'G:\Mi unidad\Mis_Pagos', anio, periodo, 'Movi.xlsx')
datos.to_excel(archivo_cp, engine='openpyxl')
Movi = datos.copy()

datos_ord = Movi.sort_values(by='Concepto', ascending=True)

In [28]:
d = Movi.loc[Movi["Monto"] <= 0, :]
debitos = d.copy()

In [8]:
debitos['Tarjetas'] = np.nan

In [10]:
debitos['Inversiones'] = np.nan

In [12]:
debitos['Tarjetas'] = debitos.loc[debitos['Descripcion'].str.contains('TARJNARANJA.+', regex=True, na=False) | debitos['Descripcion'].str.contains('VISA.+', regex=True, na=False), 'Monto']


In [14]:
debitos.loc[debitos['Descripcion'].str.contains('TARJNARANJA.+', regex=True, na=False), 'Monto'] =  np.nan 

In [23]:
debitos.loc[debitos['Descripcion'].str.contains('pepe.+', regex=True, na=False), 'Monto'] = np.nan

In [24]:
debitos.loc[debitos['Descripcion'].str.contains('pepe.+', regex=True, na=False), 'Monto'] 

Series([], Name: Monto, dtype: float64)

In [26]:
debitos['Inversiones'] = debitos.loc[debitos['Concepto'].str.contains('D.bito plazo fijo.+', regex=True, na=False) |                                                                                                debitos['Concepto'].str.contains('D.bito operaci.n.+cambio ME.+', regex=True, na=False) |                                                                                                                     debitos['Concepto'].str.contains('IMPUESTO P.A.I.S.+', regex=True, na=False) |                                                                                                                                      debitos['Concepto'].str.contains('.+GAN.+TENENCIA.+4815/20.+', regex=True, na=False), 'Monto']

In [30]:
debitos['Inversiones'] = debitos.loc[debitos['Concepto'].str.contains('D.bito plazo fijo.+', regex=True, na=False) | \
                                         debitos['Concepto'].str.contains('D.bito operaci.n.+cambio ME.+', regex=True, na=False) | \
                                         debitos['Concepto'].str.contains('IMPUESTO P.A.I.S.+', regex=True, na=False) | \
                                         debitos['Concepto'].str.contains('.+GAN.+TENENCIA.+4815/20.+', regex=True, na=False), 'Monto']
    

In [32]:
debitos

,Fecha,Concepto,Descripcion,Monto,NroComprobante,Inversiones
0,2022-02-25,Compras Maestro POSNet,VIUMI*CARNES EL FARAON,-1500.00,284329,NaN
1,2022-02-25,Compras Maestro POSNet,FRUTOS DE REY,-1078.00,470226,NaN
2,2022-02-25,TRANSF.HB.EXENTO,"23208706454-NAVAS, MARIA GABRIELA-VAR",-50000.00,473376,NaN
3,2022-02-25,Mantenimiento mundo Bancor,NaN,-1432.22,0,NaN
4,2022-02-25,IVA sobre comisión,NaN,-300.77,0,NaN
5,2022-02-22,Compras Maestro POSNet,CHICOM SA,-937.50,22209,NaN
6,2022-02-22,Compras Maestro POSNet,VIUMI*CARNES EL FARAON,-780.00,773913,NaN
7,2022-02-18,Compras Maestro POSNet,MERCADOPAGO*EQUUS,-850.00,761673,NaN
8,2022-02-18,Compras Maestro POSNet,CLEMENTE FULL SHOP,-590.00,844443,NaN
9,2022-02-18,Compras Maestro POSNet,DELICIUS,-570.00,56977,NaN


In [33]:
debitos['Tarjetas'] = debitos.loc[debitos['Descripcion'].str.contains('TARJNARANJA.+', regex=True, na=False) | \
                                      debitos['Descripcion'].str.contains('VISA.+', regex=True, na=False), 'Monto']
debitos.loc[debitos['Descripcion'].str.contains('TARJNARANJA.+', regex=True, na=False), 'Monto'] = np.nan
debitos.loc[debitos['Descripcion'].str.contains('VISA.+', regex=True, na=False), 'Monto'] = np.nan 

In [35]:
mes_deb = 'Feb_Debitos'
rango = libro.sheets(mes_deb).tables(mes_deb).data_body_range
fila = rango.get_address(row_absolute=False, column_absolute=False)[4:]
#fila_ini = str(int(fila) + 3)
#rng_ini = 'B'+ fila_ini
#libro.sheets(mes_deb).range(rng_ini).options(pd.DataFrame, expand='table').value = totales_deb

In [72]:
rango = libro.sheets(mes_deb).tables(mes_deb).data_body_range

In [73]:
rango

<Range [GGastos_v060422.xlsm]Feb_Debitos!$A$2:$J$48>

In [57]:
rango.get_address(row_absolute=False, column_absolute=False)

'A2:J48'

In [74]:
fila = rango.rows.count

In [69]:
type(fila)

int

In [75]:
fila

47

In [37]:
fila = rango.get_address(row_absolute=False, column_absolute=False)[4:]

In [39]:
fila = rango.get_address(row_absolute=False, column_absolute=False)

In [40]:
fila

'A2:J48'

In [63]:
mes_cre = 'Feb_Creditos'
rangoc = libro.sheets(mes_cre).tables(mes_cre).data_body_range

In [64]:
rangoc

<Range [GGastos_v060422.xlsm]Feb_Creditos!$A$2:$G$9>

In [52]:
rangoc.get_address(row_absolute=False, column_absolute=False)

'A2:G9'

In [53]:
rangoc.get_address(row_absolute=False, column_absolute=False)[4]

'9'

In [65]:
rangoc.rows.count

8

In [76]:
fila_ini = str(fila + 4)
rng_ini = 'B'+ fila_ini
#libro.sheets(mes_deb).range(rng_ini).options(pd.DataFrame, expand='table').value = totales_deb


In [77]:
rng_ini

'B51'

In [47]:
fila

'A2:J48'

In [45]:
fila

'48'